### 垃圾回收
- 内存泄漏: 指程序本身没有设计好，导致程序未能释放已不再使用的内存,进而失去了对这段内存的控制，从而造成了内存的浪费。
- 查看引用计数: sys.getrefcount(),如果调用那么getrefcount 本身也会引入一次计数。sys.getrefcount() 函数并不是统计一个指针，而是要统计一个对象被引用的次数，所以最后一共会有八次引用。
- 手动释放内存:
    - del a
    - gc.collect()
- 引用次数为 0 是垃圾回收启动的充分非必要条件.
- Python 使用标记清除（mark-sweep）算法和分代收集（generational），来启用针对循环引用的自动垃圾回收。
    - 在 Python 的垃圾回收实现中，mark-sweep 使用双向链表维护了一个数据结构，并且只考虑容器类的对象（只有容器类对象才有可能产生循环引用）。
    - Python 将所有对象分为三代。刚刚创立的对象是第 0 代；经过一次垃圾回收后，依然存在的对象，便会依次从上一代挪到下一代。而每一代启动自动垃圾回收的阈值，则是可以单独指定的。当垃圾回收器中新增对象减去删除对象达到相应的阈值时，就会对这一代对象启动垃圾回收。分代收集基于的思想是，新生的对象更有可能被垃圾回收，而存活更久的对象也有更高的概率继续存活。

In [4]:
import sys

a = []

# 两次引用，一次来自 a，一次来自 getrefcount
print(sys.getrefcount(a))

def func(a):
    # 三次引用，a，python 的函数调用栈，函数参数，和 getrefcount
    print(sys.getrefcount(a))

func(a)

# 两次引用，一次来自 a，一次来自 getrefcount，函数 func 调用已经不存在
print(sys.getrefcount(a))

2
3
2


In [5]:
import sys

a = []

print(sys.getrefcount(a)) # 两次

b = a

print(sys.getrefcount(a)) # 三次

c = b
d = b
e = c
f = e
g = d

print(sys.getrefcount(a)) # 八次

########## 输出 ##########

2
3
8

2
3
8


8

### 调试内存泄漏
- objgraph，一个非常好用的可视化引用关系的包。
    - show_refs()，它可以生成清晰的引用关系图。
    - show_backrefs(),有用的参数，比如层数限制（max_depth）、宽度限制（too_many）、输出格式控制（filename output）、节点过滤（filter, extra_ignore）等。

In [7]:
import objgraph

a = [1, 2, 3]
b = [4, 5, 6]

a.append(b)
b.append(a)

objgraph.show_refs([a])

Graph written to /var/folders/m2/fcsv3wxs7hs7sms1cny8x0180000gn/T/objgraph-vj7v3_bz.dot (8 nodes)
Graph viewer (xdot) and image renderer (dot) not found, not doing anything else


In [ ]:
import objgraph

a = [1, 2, 3]
b = [4, 5, 6]

a.append(b)
b.append(a)

objgraph.show_backrefs([a])